In [1]:
# regression plots

import numpy as np
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
from itertools import product
matplotlib.rcParams.update({'font.size': 13})


# a function to get data from any machine learning models and hyperparameters
def get_data(model, hyperparameters, inputs):

    # get every combination of hyperparameters without loops
    keys = list(hyperparameters)
    hyperparameters_list = list(product(*(hyperparameters[key] for key in keys)))
    # change it into dictionary
    hyperparameters_list = [dict(zip(keys, values)) for values in hyperparameters_list]
    
    y_new = np.zeros((len(inputs), len(hyperparameters_list)))
    np.random.seed(0)

    for i, hyperparameter in enumerate(hyperparameters_list):
        reg = model(**hyperparameter)
        reg.fit(X[:, np.newaxis],y)
        y_new[:, i] = reg.predict(inputs[:, np.newaxis])

    y_new = pd.DataFrame(y_new.reshape(len(inputs), len(hyperparameters_list)))
    # split every columns to lists
    tmp = []
    for i in range(len(hyperparameters_list)):
        tmp.append(y_new[i])
    # reshape
    y_new = np.array(tmp).reshape(len(inputs)*len(hyperparameters_list),1)

    return y_new


def get_title(hyperparameters):
    # get title for any number of hyperparameters
    keys = list(hyperparameters)
    title = ''
    for i in range(len(keys)):
        title += keys[i] + ' = ' + str(hyperparameters[keys[i]])[1:-1] + ', '
    return title[:-2]


def vis(X, y, model, param_grid, input):
    
    # make a copy of param_grid and get the keys
    h_temp = param_grid.copy()
    keys = list(param_grid)

    # get the sliders for each hyperparameter
    sliders = {}
    for h in range(len(param_grid)):
        param_name = keys[h]
        sliders[param_name] = widgets.SelectionSlider(options=param_grid[keys[h]], value=param_grid[keys[h]][0], description=param_name)

    # function to make the plot
    @interact(**sliders)
    def plot_series(**kwargs):
        for h in range(len(param_grid)):
            param_name = keys[h]
            slider_value = kwargs[param_name]
            h_temp[param_name] = [slider_value]
        y_new = get_data(model, h_temp, input)
        plt.figure(figsize=(6.4,3.6))
        plt.scatter(X,y,label='training data')
        plt.plot(np.linspace(0, 1, 100),true_fun(np.linspace(0, 1, 100)),label='true function')
        plt.plot(input,y_new.reshape(len(input),),'r',label='prediction')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(get_title(h_temp))
        plt.legend()

    return

In [1]:
# classification plots
import plotly
import plotly.graph_objs as go
import ipywidgets

# set step size in the mesh
h = 0.02
    
def generate_all_data(X, y, ml_algo, param_grid):
    """Returns dictionary of 2D array predicted probabilities within grid for given ML algorithm, input data, and hyperparamater grid"""
    
    # create dictionary for 2D arrays of predicted probabilities
    Z_dict = {}
    
    # create mesh grid:
    # set plot range to slightly larger than data range
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    # create meshgrid
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    np.random.seed(0)
    # loop through all hyperparameter combinations
    for p in range(len(ParameterGrid(param_grid))):
        # set current parameters
        params = ParameterGrid(param_grid)[p]
        #print('   ',params) 
        
        # set the classifier algorithm - maybe bring this outside for loop
        clf = ml_algo
        
        # set the model hyperparameters
        clf.set_params(**params)
        
        # fit the classifier on the given input data 
        clf.fit(X,y)

        # calculate predicted probabilities in mesh
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # reshape predicted probabilities
        Z = Z.reshape(xx.shape) 
        
        # convert parameter values to string key
        param_key = str(sorted(params.items()))
            
        # add 2D array to dict with param values tuple as key 
        Z_dict[param_key] = Z
        
    return Z_dict


def plot_clf_contour(param_grid, X, y):
    """Returns an interactive contour plot of predicted classification probabilities 
    and a scatterplot of input data for a given ML classifier's hyperparameter grid, 
    input data, dictionary of 2D predicted probability arrays, and grid step size."""
    # set plot range to slightly larger than data range
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
 
    Z_dict = generate_all_data(X, y, ML_algo, param_grid)

    # create figure widget
    fig = go.FigureWidget()
    # add contour plot
    fig.add_contour()
    # set plot size
    fig.update_layout(
        autosize=False,
        width=640,
        height=480)
    # add scatter plot
    fig.add_scatter(x=X[:, 0], y=X[:, 1], mode='markers')
    # add second contour line for decision boundary
    fig.add_contour()

    # create dict of sliders 
    sliders={}

    # create second dict of hyperparameters and their initial values
    slider_init_val={}

    # create sliders for each hyperparameter
    for hyperparam in range(len(param_grid)):
        # get hyperparameter name
        param_name = list(param_grid.keys())[hyperparam]
        # get hyperparameter options
        param_options = param_grid[list(param_grid.keys())[hyperparam]]
        
        # add slider with given hyperparameter name and values to dict
        # if options are categorical, create dropdown slider
        if all(isinstance(item, str) for item in param_options):
            sliders[param_name]=ipywidgets.Dropdown(options=param_options,description=param_name)
        # otherwise create selection slider
        else:
            sliders[param_name]=ipywidgets.SelectionSlider(options=param_options,description=param_name, orientation='horizontal')
        # add slider initial value to initial value dict
        slider_init_val[param_name]=param_grid[list(param_grid.keys())[hyperparam]][0]


    # create interactive plot with sliders that update hyperparameter keys and associated 2d arrays
    @interact(**sliders)
    def update(**slider_init_val):
        with fig.batch_update():
            # set contour colorscale
            fig.data[0].colorscale = "RdBu" # set red blue colorscale
            fig.data[0].contours=dict(
                size=0.05,
                start=0,
                end=1)
            fig.data[0].colorbar={"title": 'predicted probability'}
            # set contourplot data and axes 
            # set parameter key (sorted slider values)
            param_key = str(sorted(slider_init_val.items()))
            fig.data[0].z=Z_dict[param_key] # update Z - predicted prob 2D array 
            fig.data[0].x=np.arange(x_min, x_max, h)
            fig.data[0].y=np.arange(y_min, y_max, h)

            # set contour line for decision boundary data
            fig.data[2].colorscale = [[0, 'rgb(0,0,0)'], [1, 'rgb(0,0,0)']]
            fig.data[2].z=Z_dict[param_key] # update Z - predicted prob 2D array 
            fig.data[2].x=np.arange(x_min, x_max, h)
            fig.data[2].y=np.arange(y_min, y_max, h)
            fig.data[2].contours={"showlabels":False, "coloring":"lines",
                                 "start":0.5, "end":0.5}
            fig.data[2].line.width=5
            fig.data[2].ncontours=1
            # remove scale for contour line
            fig.data[2].showscale=False

            # set scatterplot point color and shape
            fig.data[1].marker.color=y
            fig.data[1].marker.colorscale=[[0, 'rgb(255,0,0)'], [1, 'rgb(0,0,255)']]
            fig.data[1].marker.size=8
            fig.data[1].marker.line.width=1


            # generate title
            title=""
            for i, (k, v) in enumerate(slider_init_val.items()):
                if i == len(slider_init_val.items())-1: # if last item, omit comma
                    title+=(str(k) + " = " + str(v))
                else:
                    title+=(str(k) + " = " + str(v)+ ", ")

            # axes labels
            fig.update_layout(
                title={
                    'text': title,
                    'y':0.9,
                    'x':0.41,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title="feature 1",
                yaxis_title="feature 2",
                font=dict(
                    family="arial, monospace",
                    size=13)
            )
    return fig

